# 如何管理代理步骤

在此示例中，我们将构建一个显式管理中间过程的 ReAct Agent
步骤。

前面的示例只是将所有消息放入模型中，但是额外的
上下文可能会分散代理的注意力并增加 API 调用的延迟。在这个例子中
我们只会在聊天历史记录中包含 `N` 最近的消息。注意
这是为了说明一般的国家管理。

## 设置

首先我们需要安装所需的包：
```bash
yarn add @langchain/langgraph @langchain/openai @langchain/core
```
接下来，我们需要为 Anthropic（我们将使用的 LLM）设置 API 密钥。

In [1]:
// process.env.OPENAI_API_KEY = "sk_...";

或者，我们可以设置 API 密钥
[LangSmith 追踪](https://smith.langchain.com/)，这将为我们提供
一流的可观测性。

In [2]:
// 可选，在 LangSmith 中添加跟踪
// process.env.LANGCHAIN_API_KEY = "ls__...";
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "Managing Agent Steps: LangGraphJS";

Managing Agent Steps: LangGraphJS


## 设置状态

`langgraph` 中图形的主要类型是
[StateGraph](/langgraphjs/reference/classes/langgraph.StateGraph.html)。
该图由一个状态对象参数化，该状态对象传递给每个
节点。然后每个节点返回操作来更新该状态。这些操作
可以设置状态的特定属性（例如覆盖现有的
值）或添加到现有属性。是否设置或添加表示在
您构建图表所用的状态对象。

对于这个例子，我们将跟踪的状态只是一个消息列表。我们
希望每个节点只将消息添加到该列表中。因此，我们将定义
声明如下：

In [3]:
import { Annotation } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

## 设置工具

我们将首先定义我们想要使用的工具。对于这个简单的例子，我们将
创建一个占位符搜索引擎。创建自己的工具真的很容易 -
查看文档
[此处](https://js.langchain.com/docs/modules/agents/tools/dynamic)了解如何操作
那。

In [4]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = new DynamicStructuredTool({
  name: "search",
  description: "Call to surf the web.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
  func: async ({}: { query: string }) => {
    // 这是一个占位符，但不要告诉法学硕士......
    return "Try again in a few seconds! Checking with the weathermen... Call be again next.";
  },
});

const tools = [searchTool];

我们现在可以将这些工具包装在一个简单的
[ToolNode](/langgraphjs/reference/classes/langgraph_prebuilt.ToolNode.html).\
这是一个简单的类，它接收包含消息的消息列表
[带有 tool_calls 的 AIMessages](https://v02.api.js.langchain.com/classes/langchain_core_messages_ai.AIMessage.html),
运行工具，并将输出返回为
[ToolMessage](https://v02.api.js.langchain.com/classes/langchain_core_messages_tool.ToolMessage.html)。

In [5]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode<typeof AgentState.State>(tools);

## 设置模型

现在我们需要加载我们想要使用的聊天模型。这应该满足两个
标准：

1.它应该与消息一起使用，因为我们的状态主要是消息列表
（聊天记录）。
2.它应该与工具调用一起使用，因为我们使用的是预构建的
[工具节点](/langgraphjs/reference/classes/langgraph_prebuilt.ToolNode.html)

**注意：** 这些模型要求不是使用 LangGraph 的要求 -
它们只是这个特定示例的要求。

In [6]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
});

In [7]:
// 完成此操作后，我们应该确保模型知道它可以调用这些工具。
// 我们可以通过将工具绑定到模型类来做到这一点。
const boundModel = model.bindTools(tools);

## 定义节点

我们现在需要在图中定义一些不同的节点。在`langgraph`中，一个节点
可以是一个函数或一个
[可运行](https://js.langchain.com/docs/expression_language/)。有两个
为此我们需要的主要节点：

1. 代理：负责决定采取什么（如果有）行动。
2. 调用工具的函数：如果代理决定采取行动，该节点
然后将执行该操作。

我们还需要定义一些边。其中一些边缘可能是有条件的。
它们是有条件的原因是基于节点的输出，其中之一
可以采取多条路径。直到该节点才知道所采取的路径
运行（LLM 决定）。

1. 条件边缘：调用代理后，我们应该：如果
代理说要采取行动，那么调用工具的函数应该是
称为\
b.如果代理说已经完成，那么就应该完成
2. 正常边缘：调用工具后，应始终返回到
代理决定下一步做什么

让我们定义节点以及一个函数来决定什么条件
边取。

In [8]:
import { END } from "@langchain/langgraph";
import { AIMessage, ToolMessage } from "@langchain/core/messages";
import { RunnableConfig } from "@langchain/core/runnables";

// 定义判断是否继续的函数
const shouldContinue = (state: typeof AgentState.State) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  // 如果没有函数调用，那么我们就完成了
  if (!lastMessage.tool_calls || lastMessage.tool_calls.length === 0) {
    return END;
  }
  // 否则，如果有，我们继续
  return "tools";
};

// **修改**
//
// 在这里，我们不会将所有消息传递给模型，而是仅传递最近的“N”消息。请注意，这是一种非常简单的处理消息的方法，仅供参考，根据您的用例，您可能还需要研究其他方法。我们还必须确保不会截断聊天历史记录以首先包含工具消息，因为这会导致 API 错误。
const callModel = async (
  state: typeof AgentState.State,
  config?: RunnableConfig,
) => {
  let modelMessages = [];
  for (let i = state.messages.length - 1; i >= 0; i--) {
    modelMessages.push(state.messages[i]);
    if (modelMessages.length >= 5) {
      if (!ToolMessage.isInstance(modelMessages[modelMessages.length - 1])) {
        break;
      }
    }
  }
  modelMessages.reverse();

  const response = await boundModel.invoke(modelMessages, config);
  // 我们返回一个对象，因为这将被添加到现有列表中
  return { messages: [response] };
};

## 定义图表

我们现在可以将它们放在一起并定义图表！

In [9]:
import { START, StateGraph } from "@langchain/langgraph";

// 定义一个新图
const workflow = new StateGraph(AgentState)
  .addNode("agent", callModel)
  .addNode("tools", toolNode)
  .addEdge(START, "agent")
  .addConditionalEdges(
    "agent",
    shouldContinue,
  )
  .addEdge("tools", "agent");

// 最后，我们编译它！
// 这会将其编译成 LangChain Runnable，
// 这意味着您可以像使用任何其他可运行程序一样使用它
const app = workflow.compile();

## 使用它！

我们现在可以使用它了！现在这暴露了
[相同的界面](https://js.langchain.com/docs/expression_language/) 与所有
其他 LangChain 运行程序。

In [10]:
import { HumanMessage, isAIMessage } from "@langchain/core/messages";
import { GraphRecursionError } from "@langchain/langgraph";

const prettyPrint = (message: BaseMessage) => {
  let txt = `[${message._getType()}]: ${message.content}`;
  if (
    (isAIMessage(message) && (message as AIMessage)?.tool_calls?.length) ||
    0 > 0
  ) {
    const tool_calls = (message as AIMessage)?.tool_calls
      ?.map((tc) => `- ${tc.name}(${JSON.stringify(tc.args)})`)
      .join("\n");
    txt += ` \nTools: \n${tool_calls}`;
  }
  console.log(txt);
};

const inputs = {
  messages: [
    new HumanMessage(
      "what is the weather in sf? Don't give up! Keep using your tools.",
    ),
  ],
};
// 设置 recursionLimit 将设置最大步数。我们希望这会无限循环:)
try {
  for await (
    const output of await app.stream(inputs, {
      streamMode: "values",
      recursionLimit: 10,
    })
  ) {
    const lastMessage = output.messages[output.messages.length - 1];
    prettyPrint(lastMessage);
    console.log("-----\n");
  }
} catch (e) {
  // 由于我们正在截断聊天记录，代理永远没有机会
  // 看到足够的信息来知道停止，所以它会一直循环，直到我们点击
  // 最大递归限制。
  if ((e as GraphRecursionError).name === "GraphRecursionError") {
    console.log("As expected, maximum steps reached. Exiting.");
  } else {
    console.error(e);
  }
}

[human]: what is the weather in sf? Don't give up! Keep using your tools.
-----

[ai]:  
Tools: 
- search({"query":"current weather in San Francisco"})
-----

[tool]: Try again in a few seconds! Checking with the weathermen... Call be again next.
-----

[ai]:  
Tools: 
- search({"query":"current weather in San Francisco"})
-----

[tool]: Try again in a few seconds! Checking with the weathermen... Call be again next.
-----

[ai]:  
Tools: 
- search({"query":"current weather in San Francisco"})
-----

[tool]: Try again in a few seconds! Checking with the weathermen... Call be again next.
-----

[ai]:  
Tools: 
- search({"query":"current weather in San Francisco"})
-----

[tool]: Try again in a few seconds! Checking with the weathermen... Call be again next.
-----

[ai]:  
Tools: 
- search({"query":"current weather in San Francisco"})
-----

As expected, maximum steps reached. Exiting.
